In [468]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import pickle

In [469]:
df=pd.read_excel("IEP_K_means.xlsx")

In [470]:
df_new=df.drop(columns=["Interface Name"],axis=1)

In [471]:
dfn_cp=df_new.copy()

In [472]:
df_new.isnull().sum()

No of Lines of code                   0
No of Components                      0
No of External Interactions           0
No of Lines of Transformation Code    0
No of Extensions                      0
Total                                 0
Complexity                            0
dtype: int64

In [473]:
df_new.dtypes

No of Lines of code                     int64
No of Components                      float64
No of External Interactions             int64
No of Lines of Transformation Code    float64
No of Extensions                        int64
Total                                 float64
Complexity                             object
dtype: object

In [474]:
encoder_list=[]
def encode_categoricaldata(name,x):
    print(x.dtype)
    le = preprocessing.LabelEncoder()
    if x.dtype=="object":
        le.fit(x)
        df_new[name] = le.transform(x)
        label_file = name+"_encoder.sav"
        pickle.dump(le, open(label_file, 'wb'))
        encoder_list.append({name:label_file})
for i in df_new:
    print(i)
    encode_categoricaldata(i,df_new[i])
    

No of Lines of code
int64
No of Components
float64
No of External Interactions
int64
No of Lines of Transformation Code
float64
No of Extensions
int64
Total
float64
Complexity
object


In [475]:
print("encoding list:",encoder_list)
file = open('Complexity_encoder.sav', 'rb')
pickled_data = pickle.load(file)
print(type(pickled_data))
print("pickled data:",pickled_data.classes_)
file.close()

encoding list: [{'Complexity': 'Complexity_encoder.sav'}]
<class 'sklearn.preprocessing.label.LabelEncoder'>
pickled data: ['High' 'Low' 'Medium']


In [476]:
df_new["Complexity"].unique()

array([1, 2, 0], dtype=int64)

In [477]:
# df_new["Complexity"]=df_new["Complexity"].map({"Low":0,"Medium":1,"High":2})

In [478]:
X=df_new.drop(columns=["Complexity"],axis=1)
y=df_new["Complexity"]

In [479]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [480]:
print(X_train.shape)

(72, 6)


In [481]:
from xgboost import XGBClassifier

In [482]:
model_XGB = XGBClassifier(learning_rate=0.1)
model_XGB.fit(X_train, y_train)

[18:43:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=100, n_jobs=8, num_parallel_tree=1,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=None, subsample=1,
       tree_method='exact', use_label_encoder=True, validate_parameters=1,
       verbosity=None)

In [483]:
y_pred_XGB = model_XGB.predict(X_test)
y_pred_XGB

array([0, 2, 1, 0, 2, 2, 1, 2, 2, 1, 0, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 0,
       2, 0, 2, 2, 2, 2, 2, 2, 2])

In [484]:
x_train_XGB = model_XGB.predict(X_train)
print("Training Accuracy of xgboost:",accuracy_score(y_train, x_train_XGB))

Training Accuracy of xgboost: 1.0


In [485]:
print("Validation Accuracy of xgboost:",accuracy_score(y_test, y_pred_XGB))

Validation Accuracy of xgboost: 1.0


In [486]:
import lightgbm as lgb

In [487]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': {'l2', 'l1'},
'num_leaves': 31,
'learning_rate': 0.05,
'feature_fraction': 0.9,
'bagging_fraction': 0.8,
'bagging_freq': 5,
'verbose': 0
}

In [488]:
model_lgbm = lgb.train(params,
lgb_train,
num_boost_round=100,
valid_sets=lgb_eval,
early_stopping_rounds=3)

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000066 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's l1: 0.689779	valid_0's l2: 0.598101
Training until validation scores don't improve for 3 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's l1: 0.683644	valid_0's l2: 0.577603
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's l1: 0.677816	valid_0's l2: 0.559641
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's l1: 0.672279	valid_0's l2: 0.543941
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's l1: 0.667019	valid_0's l2:

In [489]:
y_pred_lgbm = model_lgbm.predict(X_test, num_iteration=model_lgbm.best_iteration)

In [490]:
y_pred_lgbm

array([0.51792834, 1.76792326, 1.20932236, 0.50426232, 0.79056433,
       0.77689831, 1.17120634, 1.76792326, 1.72980723, 1.17120634,
       0.50426232, 1.20932236, 1.17120634, 1.76792326, 1.20932236,
       1.17120634, 0.79056433, 1.17120634, 1.72980723, 1.76792326,
       1.17120634, 0.51792834, 1.72980723, 0.50426232, 1.72663139,
       1.68851537, 1.35061725, 1.72980723, 1.72980723, 1.68851537,
       1.72980723])

In [491]:
predictions = [round(value) for value in y_pred_lgbm]

In [492]:
print("Validation Accuracy of lgbm:",accuracy_score(y_test, predictions))

Validation Accuracy of lgbm: 0.7096774193548387


In [493]:
from catboost import CatBoostClassifier
model_cb = CatBoostClassifier(iterations=10,
                           learning_rate=1,
                           depth=2,
                           loss_function='MultiClass')

model_cb.fit(X_train, y_train, 
        eval_set=(X_test, y_test), 
        verbose=False
)

In [494]:
y_pred_catboost = model_cb.predict(data=X_test)

In [495]:
print("Validation Accuracy of lgbm:",accuracy_score(y_test, y_pred_catboost))

Validation Accuracy of lgbm: 0.967741935483871


In [496]:
# test_data=dfn_cp.head(1)
# print(type(test_data))
# test_data

test_data={'No of Lines of code':1859,'No of Components':37.18,'No of External Interactions':1,
           'No of Lines of Transformation Code':464.75,
          'No of Extensions':4,'Total':2365.93,'Complexity':'High'}
test_data=pd.DataFrame(test_data, index=[0])
print(type(t_df))
print(t_df.columns)

<class 'pandas.core.frame.DataFrame'>
Index(['No of Lines of code', 'No of Components',
       'No of External Interactions', 'No of Lines of Transformation Code',
       'No of Extensions', 'Total'],
      dtype='object')


In [497]:
col_list=list(test_data.columns)
print(col_list)
for i in encoder_list:
    for key,value in i.items():
        if key in col_list:
            file = open(value, 'rb')
            pickled_data = pickle.load(file)
            test_data[key] = pickled_data.transform(test_data[key])
            file.close()

['No of Lines of code', 'No of Components', 'No of External Interactions', 'No of Lines of Transformation Code', 'No of Extensions', 'Total', 'Complexity']


In [498]:
test_data=test_data.drop(columns=["Complexity"],axis=1)

In [499]:
test_data

,No of Lines of code,No of Components,No of External Interactions,No of Lines of Transformation Code,No of Extensions,Total
0,1859,37.18,1,464.75,4,2365.93


In [500]:
result=model_XGB.predict(test_data)

In [501]:
result

array([1])